In [1]:
import random
import cv2
from pathlib import Path
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from torchvision import models
from torchinfo import summary
import numpy as np
import torchmetrics
from src.data.dataset import ProcessedWoundDataset
from src.data.make_dataset import make_dataset1, make_dataset2
from src.utils import empty_directory
from src.models.mobilnetv2 import MobileNetV2withDecoder
from config.defaults import get_cfg_defaults

In [9]:
import itertools

def generate_unique_pairs(people):
    pairs = set()
    for i, person in enumerate(people):
        other_people = people[:i] + people[i+1:]
        for other_person in other_people:
            pair = tuple(sorted((person, other_person)))
            pairs.add(pair)
    return sorted(pairs, key=lambda x: (x[0], x[1]))

# Example usage:
people = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J"]
unique_pairs = generate_unique_pairs(people)
print("Unique pairs:")
for pair in unique_pairs:
    print("-".join(pair))


Unique pairs:
A-B
A-C
A-D
A-E
A-F
A-G
A-H
A-I
A-J
B-C
B-D
B-E
B-F
B-G
B-H
B-I
B-J
C-D
C-E
C-F
C-G
C-H
C-I
C-J
D-E
D-F
D-G
D-H
D-I
D-J
E-F
E-G
E-H
E-I
E-J
F-G
F-H
F-I
F-J
G-H
G-I
G-J
H-I
H-J
I-J
